In [2]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna

d:\LUIS PP\ULPGC\TERCERO\AA2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Definir las transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionar las imágenes a 128x128
    transforms.ToTensor(),          # Convertir las imágenes a tensores
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar con media 0.5 y desviación estándar 0.5
])

# Especificar la ruta del dataset
dataset_path = 'Flags'  # Cambia esto a la ruta donde descargaste el dataset

# Cargar el dataset de imágenes
train_data = datasets.ImageFolder(root=dataset_path, transform=transform)

# Calcular el tamaño total del conjunto de datos
dataset_size = len(train_data)

# Definir las proporciones para dividir el dataset en entrenamiento y prueba
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

# Dividir el dataset en subconjuntos de entrenamiento y prueba
train_dataset, test_dataset = random_split(train_data, [train_size, test_size])

# Crear DataLoaders para los conjuntos de entrenamiento y prueba
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Obtener los nombres de las clases
class_names = train_data.classes

# Imprimir información sobre el conjunto de datos
print(f'Clases encontradas: {class_names}')
print(f'Número de clases: {len(class_names)}')
print(f'Tamaño del conjunto de entrenamiento: {len(train_loader.dataset)}')
print(f'Tamaño del conjunto de prueba: {len(test_loader.dataset)}')

Clases encontradas: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Holland', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Slovakia', 'Slovenia', 'South Cyprus', 'Spain', 'Sweden']
Número de clases: 24
Tamaño del conjunto de entrenamiento: 796
Tamaño del conjunto de prueba: 200


In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        # Capa de convolución 1: Entrada de 3 canales (RGB), salida de 16 canales
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)

        # Max Pooling: Kernel 2x2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Capa de convolución 2: Entrada de 16 canales, salida de 32 canales
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)

        # Capa totalmente conectada 1: Después de dos capas de convolución y pooling, el tamaño es 32x32
        # Imágenes de entrada de 128x128 reducidas a 32x32 por el pooling
        self.fc1 = nn.Linear(32 * 32 * 32, 120)

        # Capa totalmente conectada 2: Reducción a 24 salidas para las 3 clases (Rock, Paper, Scissors)
        self.fc2 = nn.Linear(120, 24)

    def forward(self, x):
        # Primera capa de convolución seguida de ReLU y pooling
        x = F.relu(self.conv1(x))  # Activación ReLU después de la convolución
        x = self.pool(x)  # Pooling

        # Segunda capa de convolución seguida de ReLU y pooling
        x = F.relu(self.conv2(x))  # Activación ReLU después de la convolución
        x = self.pool(x)  # Pooling

        # Aplanar la salida para pasar a la capa totalmente conectada
        x = x.view(-1, 32 * 32 * 32)  # Aplanamos la salida

        # Primera capa totalmente conectada
        x = F.relu(self.fc1(x))

        # Segunda capa totalmente conectada (capa de salida)
        x = self.fc2(x)

        return x

In [5]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = ConvNet()  # Instanciamos el modelo de red neuronal
criterion = nn.CrossEntropyLoss()  # Usamos la pérdida de entropía cruzada para la clasificación multi-clase
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizador Adam con tasa de aprendizaje de 0.001

# Definir el número de épocas
epochs = 5

# Entrenamiento
for epoch in range(epochs):
    running_loss = 0.0  # Inicializamos la pérdida acumulada de la época

    # Iterar sobre el conjunto de entrenamiento
    for images, labels in train_loader:
        optimizer.zero_grad()  # Limpiar los gradientes previos para no acumularlos

        # Pasamos las imágenes por el modelo
        outputs = model(images)

        # Calculamos la pérdida entre las predicciones y las etiquetas
        loss = criterion(outputs, labels)

        # Backpropagation: calcular los gradientes de la pérdida con respecto a los parámetros del modelo
        loss.backward()

        # Actualizamos los pesos del modelo con el optimizador
        optimizer.step()

        # Acumulamos la pérdida para esta iteración
        running_loss += loss.item()

    # Promedio de la pérdida por época y mostrar el progreso
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

Epoch 1, Loss: 2.7526
Epoch 2, Loss: 1.3150
Epoch 3, Loss: 0.7107
Epoch 4, Loss: 0.3958
Epoch 5, Loss: 0.2423


In [6]:
# Variables para almacenar los resultados
correct = 0
total = 0

# Evaluación del modelo sin calcular gradientes
with torch.no_grad():  # Deshabilita el cálculo de gradientes, útil para la evaluación
    for images, labels in test_loader:
        outputs = model(images)  # Pasamos las imágenes por el modelo
        _, predicted = torch.max(outputs.data, 1)  # Obtenemos las predicciones (clase con mayor probabilidad)
        
        total += labels.size(0)  # Incrementamos el total de imágenes evaluadas
        correct += (predicted == labels).sum().item()  # Incrementamos el número de predicciones correctas

# Calculamos la precisión y la mostramos
accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


Accuracy of the model on the test images: 82.50%


In [7]:
# Función para definir el modelo
def create_model(trial):
    model = ConvNet()

    # Hiperparámetros que se ajustarán
    learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    return model, optimizer

# Función de entrenamiento
def train_model(trial):
    model, optimizer = create_model(trial)
    criterion = nn.CrossEntropyLoss()

    # Hiperparámetros
    batch_size = trial.suggest_int('batch_size', 16, 128, step=16)  # Tamaño de lote
    epochs = 5

    # Crear DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Evaluar el modelo
    accuracy = evaluate_model(model, test_loader)

    return accuracy  # Devuelve la métrica que se quiere optimizar

# Función para evaluar el modelo
def evaluate_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total  # Devuelve la precisión

# Crear un estudio y optimizar
study = optuna.create_study(direction='maximize')  # Queremos maximizar la precisión
study.optimize(train_model, n_trials=7)

[I 2024-12-26 16:47:25,567] A new study created in memory with name: no-name-03405a99-d61d-4cef-bb35-3cbf76e44752
C:\Users\INES\AppData\Local\Temp\ipykernel_10856\1064966657.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-1)
[I 2024-12-26 16:48:31,428] Trial 0 finished with value: 0.765 and parameters: {'lr': 0.0015744600836828603, 'batch_size': 112}. Best is trial 0 with value: 0.765.
[I 2024-12-26 16:49:35,407] Trial 1 finished with value: 0.055 and parameters: {'lr': 0.006997943703606682, 'batch_size': 32}. Best is trial 0 with value: 0.765.
[I 2024-12-26 16:50:41,295] Trial 2 finished with value: 0.86 and parameters: {'lr': 0.0008729529045138773, 'batch_size': 48}. Best is trial 2 with value: 0.86.
[I 2024-12-26 16:51:52,224] Trial 3 finished with value: 0.51 and p